In [4]:
import cv2
import tensorflow as tf
from sklearn import datasets, model_selection, metrics
import trimesh
import open3d as o3d
import numpy as np
from tensorflow import keras



In [44]:
import os

# Pedir el nombre del objeto
object_name = input("Introduce el nombre del objeto: ")

# Crear la carpeta si no existe
if not os.path.exists(object_name):
    os.makedirs(object_name)

# Iniciar la cámara
cap = cv2.VideoCapture(0)

image_count = len(os.listdir(object_name))  # Contar el número de imágenes existentes en la carpeta

while True:
    # Capturar frame por frame
    ret, frame = cap.read()
    
    # Mostrar el frame resultante
    cv2.imshow('Frame', frame)
    
    # Verificar si se presionó la tecla 'e' para guardar las imágenes
    if cv2.waitKey(1) & 0xFF == ord('e'):
        for i in range(15):  # Tomar 5 fotos
            ret, frame = cap.read()
            cv2.imwrite(f"{object_name}/image_{image_count}.jpg", frame)
            image_count += 1
    
    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()
